In [ ]:
import os
import time
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import datetime
import math
import sys

!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Asia/Seoul /etc/localtime

In [ ]:
!pip install torch-lr-finder
# import torch
from PIL import Image
# import os
import torchvision
# import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.models as models
import albumentations as A
import cv2
# import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# from torch_lr_finder import *
# from torch_lr_finder import LRFinder
# import matplotlib.pyplot as plt
import torchvision.models as models
# from sklearn.model_selection import train_test_split
from numpy import asarray 

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content

/content


In [ ]:
folderpath = "ckpt/"
if not os.path.exists(folderpath):
    os.mkdir(folderpath)

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import to_categorical

import numpy as np

In [ ]:
class imgDataset():
    def __init__(self, root, transform, test=False):
          self.root = root
          self.test = test
          self.transform = transform
          self.basic_transform = transforms.ToTensor()
          self.pFiles = glob.glob(f"{root}/0/*.jpg")
          self.pLen = len(self.pFiles)
          self.nFiles = glob.glob(f"{root}/1/*.jpg")
          self.nLen = len(self.nFiles)
  
    def __len__(self):
          return self.pLen + self.nLen
    
    def __getitem__(self, index):
          if index < self.pLen:
              image = Image.open(self.pFiles[index])
              label = 0
          else:
              image = Image.open(self.nFiles[index - self.pLen])
              label = 1

          if self.test == False:
              image = self.transform(image)
          else:
              image = self.basic_transform(image)
          if self.test == False:
              return (image, label)
          else:
              return image

In [ ]:
import glob
train_root = '/content/drive/MyDrive/Colab Notebooks/CNN_Project/datasets/train'
trainset = imgDataset(train_root,
                      transform = transforms.Compose([
                                                      transforms.ToTensor()
                      ]))
train_loader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers = 4)

valid_root = '/content/drive/MyDrive/Colab Notebooks/CNN_Project/datasets/valid'
validset = imgDataset(valid_root,
                      transform = transforms.Compose([
                                                      transforms.ToTensor()
                      ]))
valid_loader = DataLoader(validset, batch_size=64, shuffle=True, num_workers=4)

In [ ]:
model = torchvision.models.resnet101(pretrained=True)

In [ ]:
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2, bias=True)
model = model.cuda()

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
optim = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.01)

In [ ]:
def calc_accuracy(prediction, label):
    with torch.no_grad():
        _, pred2label = prediction.max(dim=1)
        same = (pred2label == label).float()
        accuracy = same.sum().item() / same.numel()
    return accuracy
epochs = 20
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, 'min')

In [ ]:
start = time.time()
best_accuracy = 0.0
for epoch in range(epochs):

    train_accuracy_list, valid_accuracy_list = [], []
    train_total_loss, valid_total_loss = 0.0, 0.0

    model.train()
    # load mini-batches and do training
    for _, (batch, label) in enumerate(train_loader):
        
        batch = batch.cuda()
        label = label.cuda()   

        prediction = model(batch)
        loss = criterion(prediction, label)

        optim.zero_grad()
        loss.backward()
        optim.step()

        train_accuracy_list.append(calc_accuracy(prediction, label))
        train_total_loss += loss.item()
    
    train_total_accuracy = sum(train_accuracy_list) / len(train_accuracy_list)

    # for validation, you do not need to calculate the gradient
    with torch.no_grad():
        model.eval()
        # load mini-batches and do validation
        for _, (batch, label) in enumerate(valid_loader):
            batch = batch.cuda()
            label = label.cuda()

            prediction = model(batch)
            loss = criterion(prediction, label)
            
            valid_accuracy_list.append(calc_accuracy(prediction, label))
            valid_total_loss += loss.item()
        
        valid_total_accuracy = sum(valid_accuracy_list) / len(valid_accuracy_list)

        if valid_total_accuracy > best_accuracy:
            best_accuracy = valid_total_accuracy
            best_model_state = model.state_dict()
            best_optim_state = optim.state_dict()
    scheduler.step(valid_total_loss)
    if epoch % (epochs // 10) == 0 or epoch == epochs - 1:
        print(f"""{time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())} || [{epoch}/{epochs}], train_loss = {train_total_loss:.4f}, train_accuracy = {train_total_accuracy:.2f}, valid_loss = {valid_total_loss:.4f}, valid_accuracy = {valid_total_accuracy:.2f}""")
        print("learning rate " + str(optim.param_groups[0]['lr']))
elapsed = time.time() - start
print(f"End of training, elapsed time : {elapsed // 60} min {elapsed % 60} sec.")

2021-02-15 02:02:27 || [0/20], train_loss = 16.5288, train_accuracy = 0.74, valid_loss = 1.4998, valid_accuracy = 0.88
learning rate 0.001
2021-02-15 02:03:11 || [2/20], train_loss = 4.9922, train_accuracy = 0.95, valid_loss = 1.3119, valid_accuracy = 0.85
learning rate 0.001
2021-02-15 02:03:55 || [4/20], train_loss = 1.7159, train_accuracy = 0.99, valid_loss = 0.8070, valid_accuracy = 0.94
learning rate 0.001
2021-02-15 02:04:39 || [6/20], train_loss = 0.8489, train_accuracy = 1.00, valid_loss = 0.8599, valid_accuracy = 0.95
learning rate 0.001
2021-02-15 02:05:23 || [8/20], train_loss = 0.4816, train_accuracy = 1.00, valid_loss = 0.9320, valid_accuracy = 0.93
learning rate 0.001
2021-02-15 02:06:07 || [10/20], train_loss = 0.2436, train_accuracy = 1.00, valid_loss = 0.9618, valid_accuracy = 0.94
learning rate 0.001
2021-02-15 02:06:51 || [12/20], train_loss = 0.2133, train_accuracy = 1.00, valid_loss = 1.4753, valid_accuracy = 0.91
learning rate 0.001
2021-02-15 02:07:35 || [14/20],

In [ ]:
savepath = '/content/drive/MyDrive/Colab Notebooks/CNN_Project/saved_model'
if best_model_state is not None and best_optim_state is not None:
    torch.save(best_model_state, f"{savepath}/model_state_dict.pt")
    torch.save(best_optim_state, f"{savepath}/optim_state_dict.pt")
    model.load_state_dict(best_model_state)
    torch.save(model, f"{savepath}/best_model.pt")
    print("Successfully saved.")

Successfully saved.
